* Reference: https://radimrehurek.com/gensim/models/fasttext.html

TF-IDF 的全名為 Term Frequency - Inverted Document Frequency，可以看出其實是由兩個部分組成。
分別為『 Term Frequency(詞頻) 』、『  Inverted Document Frequency (倒文件頻率) 』。
舉個例子，假設有一個訓練集包含 1000 個評論，我們要預測評論是正向或負面：
其中有一個單詞在每個評論中幾乎都有出現，此時這個詞有很大的機會對我們的預測沒有顯著的影響(ex. the)
另外又有一個單詞，他在 1000 個評論中出現的頻率較低，然而卻在特定幾個評論中頻繁的出現，
此時這個單詞就可能有較高的機率對判定評論的正負面有影響(ex. excellent)。
由上述例子我們可以發現，不同單詞對一句話或一段文章的重要性並不相同，而TF-IDF的出現就是為了凸顯不同單詞的重要性。

詞頻(TF)：
一個單詞出現在一個文件的次數/該文件中所有單詞的數量
因此當一個單詞集中出現在數個評論中時，這個單詞在這些文件中的TF值就會較高
IDF :  
Log (所有文件的數目/包含這個單詞的文件數目)
IDF :  log(N/dfx) = logN - logdfx
當一個單詞集中出現在數個評論中時，此時雖然 logN 值對於所有單詞都是相同的，但這個詞的 logdfx 值會較低，因此 IDF 值也會較高。

綜合上述，當一個單詞的 TF * IDF 值越大時，代表這個單詞對整段文章的重要性也越大，我們可以歸納出：

不同單詞在同一個文章中獲得的 TFIDF 值可能不相同，值的高低代表了單詞對整段文章的重要性。
同一個單詞在不同文章所得到的 TFIDF 值也可能不同。

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

X_train = ['This is the first document, we can put another document here.', 'That is the second document.','No comment']
X_test = ['This is the third document.']

# max_df=0.5 單詞如果在50%以上的文件出現就不考慮
# min_df=5 單詞如果出現次數少於5次就不考慮, min_df=0.1 單詞如果在10%以下的文件中出現就不考慮
vectorizer = TfidfVectorizer() 
## 用X_train來build 字典，字數還有document 數量
vectorizer.fit(X_train)

## 得到tfidf vector
tfidf_train = vectorizer.transform(X_train)
tfidf_test = vectorizer.transform(X_test)

print(tfidf_test.toarray())
print('\n')
print('單詞對應index: \n{}'.format(vectorizer.vocabulary_))

[[0.         0.         0.         0.45985353 0.         0.
  0.45985353 0.         0.         0.         0.         0.45985353
  0.60465213 0.        ]]


單詞對應index: 
{'this': 12, 'is': 6, 'the': 11, 'first': 4, 'document': 3, 'we': 13, 'can': 1, 'put': 8, 'another': 0, 'here': 5, 'that': 10, 'second': 9, 'no': 7, 'comment': 2}


In [7]:
vectorizer

TfidfVectorizer()

---

#### 我們也可以將TfidfVectorizer拆解成兩個步驟：CountVectorizer, TfidfTransformer

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [9]:
#文本
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
]


#### CountVectorizer是通過fit_transform函數將文本中的詞語轉換為詞頻矩陣
* get_feature_names()可看到所有文本的關鍵字
* vocabulary_可看到所有文本的關鍵字和其位置
* toarray()可看到詞頻矩陣的結果

In [10]:
#CountVectorizer 就是 tokenlizer
vectorizer = CountVectorizer()
count = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())  
print(vectorizer.vocabulary_)
print(count.toarray())



['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
{'this': 8, 'is': 3, 'the': 6, 'first': 2, 'document': 1, 'second': 5, 'and': 0, 'third': 7, 'one': 4}
[[0 1 1 1 0 0 1 0 1]
 [0 1 0 1 0 2 1 0 1]
 [1 0 0 0 1 0 1 1 0]
 [0 1 1 1 0 0 1 0 1]]


#### TfidfTransformer是統計CountVectorizer中每個詞語的tf-idf權值

In [11]:
transformer = TfidfTransformer()
tfidf_matrix = transformer.fit_transform(count)
print(tfidf_matrix.toarray())

[[0.         0.43877674 0.54197657 0.43877674 0.         0.
  0.35872874 0.         0.43877674]
 [0.         0.27230147 0.         0.27230147 0.         0.85322574
  0.22262429 0.         0.27230147]
 [0.55280532 0.         0.         0.         0.55280532 0.
  0.28847675 0.55280532 0.        ]
 [0.         0.43877674 0.54197657 0.43877674 0.         0.
  0.35872874 0.         0.43877674]]
